#### Get Movies Type

In [19]:
import requests
import bs4
import pandas as pd
import json
from tqdm import tqdm
import copy
import random
from urllib import request
from http import cookiejar

In [3]:
def get_moviesLink_by_type(Type, types_num, header, data_csv, item_epoch, max_epoch):
    print(Type)
    # header = {
    # 'User-Agent':r'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
    with tqdm(total=400 * 16, desc=f'Epoch {item_epoch + 1}/{max_epoch}') as pbar:
        for range_item in range(16):
            for item_start in range(20):
                type_url = 'https://movie.douban.com/j/chart/top_list?type={}&interval_id={}%3A{}&action=&start={}&limit=20'.\
                    format(types_num, 100 - range_item*5, 90 - range_item*5, item_start*20)
                req = json.loads(requests.get(type_url, headers=header, timeout=50).text)
                for detail_item in req:
                    data_csv.loc[len(data_csv)] = detail_item
                    pbar.update(1)

In [11]:
header = {
    "User-Agent":r"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50"
}
req = requests.get(r"https://movie.douban.com/chart",
                   headers=header, timeout=5).text.encode("utf-8")
soup_type = bs4.BeautifulSoup(req, "html.parser").find_all(name="div", attrs={'class':'types'})[0].find_all(name='a')
data_csv = pd.DataFrame(columns=['types', 'rating', 'rank', 'id', 'regions', 'title', 'url', 'release_date', 'actor_count', 'vote_count',
                                    'score', 'actors', 'cover_url', 'is_watched'])
for item_epoch, soup_type_item in enumerate(soup_type):
    str_type = str(soup_type_item['href'])[str(soup_type_item['href']).index('type=') + 5:]
    str_type = str_type[:str_type.index("&")]
    get_moviesLink_by_type(str(soup_type_item.string), str_type, header, data_csv, item_epoch, len(soup_type))

剧情


Epoch 0/29:   1%|▏         | 80/6400 [00:06<08:27, 12.45it/s] 


KeyboardInterrupt: 

In [2]:
data_csv.to_csv("./data_movies.csv", index=False)
data_csv

NameError: name 'data_csv' is not defined

####  去重

In [3]:
data_csv = pd.read_csv("./data_movies.csv")
data_csv = data_csv.loc[data_csv['id'].astype(str).drop_duplicates().index]
print(len(data_csv.url.unique()))
print(len(data_csv.title.unique()))
print(len(data_csv.id.unique()))
print(len(data_csv))
data_csv.index = range(len(data_csv))
data_csv.to_csv("./data_movies_noRepeat.csv", index=False)
data_csv

23422
22385
23422
23422


,types,rating,rank,id,regions,title,url,release_date,actor_count,vote_count,score,actors,cover_url,is_watched
0,"['犯罪', '剧情']","['9.7', '50']",1,1292052,['美国'],肖申克的救赎,https://movie.douban.com/subject/1292052/,1994-09-10,25,2604521,9.7,"['蒂姆·罗宾斯', '摩根·弗里曼', '鲍勃·冈顿', '威廉姆·赛德勒', '克兰西·...",https://img2.doubanio.com/view/photo/s_ratio_p...,False
1,"['剧情', '爱情', '同性']","['9.6', '50']",2,1291546,"['中国大陆', '中国香港']",霸王别姬,https://movie.douban.com/subject/1291546/,1993-07-26,26,1934038,9.6,"['张国荣', '张丰毅', '巩俐', '葛优', '英达', '蒋雯丽', '吴大维',...",https://img3.doubanio.com/view/photo/s_ratio_p...,False
2,"['剧情', '喜剧', '爱情', '战争']","['9.6', '50']",3,1292063,['意大利'],美丽人生,https://movie.douban.com/subject/1292063/,2020-01-03,29,1202892,9.6,"['罗伯托·贝尼尼', '尼可莱塔·布拉斯基', '乔治·坎塔里尼', '朱斯蒂诺·杜拉诺'...",https://img2.doubanio.com/view/photo/s_ratio_p...,False
3,"['剧情', '历史', '战争']","['9.6', '50']",4,1295124,['美国'],辛德勒的名单,https://movie.douban.com/subject/1295124/,1993-11-30,47,1003257,9.6,"['连姆·尼森', '本·金斯利', '拉尔夫·费因斯', '卡罗琳·古多尔', '乔纳森·...",https://img2.doubanio.com/view/photo/s_ratio_p...,False
4,"['剧情', '犯罪', '悬疑']","['9.6', '50']",5,1296141,['美国'],控方证人,https://movie.douban.com/subject/1296141/,1957-12-17,51,453853,9.6,"['泰隆·鲍华', '玛琳·黛德丽', '查尔斯·劳顿', '爱尔莎·兰切斯特', '约翰·...",https://img1.doubanio.com/view/photo/s_ratio_p...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23417,"['黑色电影', '犯罪', '惊悚']","['7.3', '40']",79,1302943,['美国'],合约杀手,https://movie.douban.com/subject/1302943/,1942-05-13,5,479,7.3,"['维罗妮卡·莱克', '罗伯特·普雷斯顿', '拉尔德·克雷加', '艾伦·拉德', '塔...",https://img9.doubanio.com/view/photo/s_ratio_p...,False
23418,"['犯罪', '剧情', '黑色电影', '恐怖']","['7.3', '40']",80,2033530,['美国'],涨潮小屋,https://movie.douban.com/subject/2033530/,1950-03-25,24,458,7.3,"['路易斯·海沃德', '李·鲍曼', '简·维亚特', '多萝西·帕特里克', '安·休梅...",https://img1.doubanio.com/view/photo/s_ratio_p...,False
23419,"['动画', '恐怖', '黑色电影']","['7.3', '40']",81,26877383,['日本'],成人童话系列1 令人恐怖的格林童话,https://movie.douban.com/subject/26877383/,2000-06-21,0,367,7.3,[],https://img2.doubanio.com/view/photo/s_ratio_p...,False
23420,"['黑色电影', '惊悚']","['7.2', '40']",82,1293416,['美国'],逃狱雪冤,https://movie.douban.com/subject/1293416/,1947-09-27,2,607,7.2,"['亨弗莱·鲍嘉', '劳伦·白考尔']",https://img9.doubanio.com/view/photo/s_ratio_p...,False


#### 爬取详情页面（失败）

In [5]:
data_csv = pd.read_csv("D:\edge\data_movies.csv")
data_csv

,types,rating,rank,id,regions,title,url,release_date,actor_count,vote_count,score,actors,cover_url,is_watched,author,director
0,"['犯罪', '剧情']","['9.7', '50']",1,1292052,['美国'],肖申克的救赎,https://movie.douban.com/subject/1292052/,1994-09-10,25,2604521,9.7,"['蒂姆·罗宾斯', '摩根·弗里曼', '鲍勃·冈顿', '威廉姆·赛德勒', '克兰西·...",https://img2.doubanio.com/view/photo/s_ratio_p...,False,弗兰克·德拉邦特 Frank Darabont/斯蒂芬·金 Stephen King,弗兰克·德拉邦特 Frank Darabont
1,"['剧情', '爱情', '同性']","['9.6', '50']",2,1291546,"['中国大陆', '中国香港']",霸王别姬,https://movie.douban.com/subject/1291546/,1993-07-26,26,1934038,9.6,"['张国荣', '张丰毅', '巩俐', '葛优', '英达', '蒋雯丽', '吴大维',...",https://img3.doubanio.com/view/photo/s_ratio_p...,False,芦苇 Wei Lu/李碧华 Lilian Lee Pik-Wah,陈凯歌 Kaige Chen
2,"['剧情', '喜剧', '爱情', '战争']","['9.6', '50']",3,1292063,['意大利'],美丽人生,https://movie.douban.com/subject/1292063/,2020-01-03,29,1202892,9.6,"['罗伯托·贝尼尼', '尼可莱塔·布拉斯基', '乔治·坎塔里尼', '朱斯蒂诺·杜拉诺'...",https://img2.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN
3,"['剧情', '历史', '战争']","['9.6', '50']",4,1295124,['美国'],辛德勒的名单,https://movie.douban.com/subject/1295124/,1993-11-30,47,1003257,9.6,"['连姆·尼森', '本·金斯利', '拉尔夫·费因斯', '卡罗琳·古多尔', '乔纳森·...",https://img2.doubanio.com/view/photo/s_ratio_p...,False,托马斯·肯尼利 Thomas Keneally/斯蒂文·泽里安 Steven Zaillian,史蒂文·斯皮尔伯格 Steven Spielberg
4,"['剧情', '犯罪', '悬疑']","['9.6', '50']",5,1296141,['美国'],控方证人,https://movie.douban.com/subject/1296141/,1957-12-17,51,453853,9.6,"['泰隆·鲍华', '玛琳·黛德丽', '查尔斯·劳顿', '爱尔莎·兰切斯特', '约翰·...",https://img1.doubanio.com/view/photo/s_ratio_p...,False,阿加莎·克里斯蒂 Agatha Christie/比利·怀尔德 Billy Wilder/哈...,比利·怀尔德 Billy Wilder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23417,"['黑色电影', '犯罪', '惊悚']","['7.3', '40']",79,1302943,['美国'],合约杀手,https://movie.douban.com/subject/1302943/,1942-05-13,5,479,7.3,"['维罗妮卡·莱克', '罗伯特·普雷斯顿', '拉尔德·克雷加', '艾伦·拉德', '塔...",https://img9.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN
23418,"['犯罪', '剧情', '黑色电影', '恐怖']","['7.3', '40']",80,2033530,['美国'],涨潮小屋,https://movie.douban.com/subject/2033530/,1950-03-25,24,458,7.3,"['路易斯·海沃德', '李·鲍曼', '简·维亚特', '多萝西·帕特里克', '安·休梅...",https://img1.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN
23419,"['动画', '恐怖', '黑色电影']","['7.3', '40']",81,26877383,['日本'],成人童话系列1 令人恐怖的格林童话,https://movie.douban.com/subject/26877383/,2000-06-21,0,367,7.3,[],https://img2.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN
23420,"['黑色电影', '惊悚']","['7.2', '40']",82,1293416,['美国'],逃狱雪冤,https://movie.douban.com/subject/1293416/,1947-09-27,2,607,7.2,"['亨弗莱·鲍嘉', '劳伦·白考尔']",https://img9.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN


In [ ]:
us = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50",
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50']

proxy_list = [{"https": "171.35.163.80:9999"}, {'https': '182.34.36.242:9999'}, {"https": "210.22.176.146:32153"}, {"https": "171.13.137.81:9999"},
              {"https": "27.43.187.171:9999"}, {"https": "27.159.167.154:9999"}, {"https": "171.35.166.35:9999"}, {"https": "60.205.229.126:80"},
              {"https": "27.204.87.254:9999"}, {"https": "218.91.94.202:9999"}, {"https": "110.86.137.213:9999"}, {"https": "218.91.94.130:9999"}]
headers = { "User-Agent":random.choice(us) }
with tqdm(total=len(data_csv), postfix=dict) as pbar:
    for item in range(len(data_csv)):
        try:
            soup_detail = bs4.BeautifulSoup(requests.get(data_csv.loc[item, 'url'],
                                                         headers=headers, timeout=15,
                                                         proxies=random.choice(proxy_list)).text.encode("utf-8"),
                                            "html.parser").find(name='script', attrs={'type':"application/ld+json"})
            json_detail = json.loads(soup_detail.string)
            author = ''
            for item_author in json_detail['author']:
                author = author + '/' + item_author['name']
            data_csv.loc[item, 'author'] = copy.copy(author[1:])
            director = ''
            for item_director in json_detail['director']:
                director = director + '/' + item_director['name']
            data_csv.loc[item, 'director'] = copy.copy(director[1:])
            pbar.set_postfix(**{'director: ': copy.copy(director[1:]), 'author': copy.copy(author[1:])})
            pbar.update(1)
        except Exception as e:
            print(e.args)
            pbar.update(1)
        break

#### 添加导演信息

In [ ]:
data_csv_noRepeat = pd.read_csv("./data_movies_noRepeat.csv")
us = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50",
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50']
type_list = ['%E5%89%A7%E6%83%85','%E5%96%9C%E5%89%A7','%E5%8A%A8%E4%BD%9C',
              '%E7%88%B1%E6%83%85','%E7%A7%91%E5%B9%BB','%E5%8A%A8%E7%94%BB',
              '%E6%82%AC%E7%96%91','%E6%83%8A%E6%82%9A','%E6%81%90%E6%80%96',
              '%E7%8A%AF%E7%BD%AA','%E5%90%8C%E6%80%A7','%E9%9F%B3%E4%B9%90',
              '%E6%AD%8C%E8%88%9E','%E4%BC%A0%E8%AE%B0','%E5%8E%86%E5%8F%B2',
              '%E6%88%98%E4%BA%89','%E8%A5%BF%E9%83%A8','%E5%A5%87%E5%B9%BB',
              '%E5%86%92%E9%99%A9','%E7%81%BE%E9%9A%BE','%E6%AD%A6%E4%BE%A0',
              '%E6%83%85%E8%89%B2']
proxy_list = [{'https': '10.3.4.9:49300'}, {'https': '10.3.4.9:63112'}]
headers = { "User-Agent":random.choice(us)}
movies = 0
with tqdm(total=len(type_list) * 320 * 20, postfix=dict) as pbar:
    for item_type in type_list:
        for item_start in range(320):
            try:
                requests.adapters.DEFAULT_RETRIES = 5 # 增加重连次数
                s = requests.session()
                s.keep_alive = False # 关闭多余连接
                url = 'https://movie.douban.com/j/new_search_subjects?sort=S&range=0,10&tags=%E7%94%B5%E5%BD%B1&start={}&genres={}'.format(item_start * 20, item_type)
                req = json.loads(requests.get(url, headers=headers, timeout=50, proxies=random.choice(proxy_list)).text)
                for req_item in req['data']:
                    if len(data_csv_noRepeat.loc[data_csv_noRepeat['url'] == req_item['url']]) != 0:
                        try:
                            data_csv_noRepeat.loc[data_csv_noRepeat['url'] == req_item['url'], 'directors'] = str(req_item['directors'])
                        except Exception as e:
                            print(str(req_item['directors']))
                            print(data_csv_noRepeat.loc[data_csv_noRepeat['url'] == req_item['url'], 'directors'])
                        movies += 1
                    pbar.update(1)
                    pbar.set_postfix(**{'movies_num': movies})
            except Exception as E:
                print(req)

#### 通过百度百科爬取更多信息

In [54]:
def no_sup(soup):
    [s.extract() for s in soup.find_all("sup")]
    [s.extract() for s in soup.find_all("a", {'class':'sup-anchor'})]
    return soup

In [89]:
def get_item(soup):
    result = []
    for item in soup:
        if str(item.string).replace("\n", '') != '' and str(item.string).replace("\n", '') != '、' \
                and str(item.string).replace("\n", '') is not None:
            result.append(str(item.string).replace("\n", '').replace('，', ''))
            return result if len(result) > 1 else result[0]
        else:
            return None

In [97]:
data_csv_noRepeat = pd.read_csv("./data_movies_noRepeat.csv")
us = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 "
      "Safari/537.36 Edg/100.0.1185.50",
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50']
headers = { "User-Agent":random.choice(us)}
movies = 0
with tqdm(total=len(data_csv_noRepeat), postfix=dict) as pbar:
    for item in range(len(data_csv_noRepeat)):
        try:
            requests.adapters.DEFAULT_RETRIES = 5 # 增加重连次数
            s = requests.session()
            s.keep_alive = False # 关闭多余连接
            url = 'https://baike.baidu.com/item/{}'.format(data_csv_noRepeat.loc[item, 'title'])
            req = requests.get(url, headers=headers, timeout=5).text
            soup_detail = bs4.BeautifulSoup(req, "html.parser").find_all(name='div',
                                                                         attrs={'class':'basic-info J-basic-info '
                                                                                        'cmn-clearfix'})[0]
            soup_detail_name = soup_detail.find_all(name='dt', attrs={'class':'basicInfo-item name'})
            soup_detail_value = soup_detail.find_all(name='dd', attrs={'class':'basicInfo-item value'})
            if len(soup_detail_value) == len(soup_detail_name):
                for item_detail in range(len(soup_detail_value)):
                    j = 0
                    name_str = str(soup_detail_name[item_detail].string).replace(u'\xa0', '')
                    soup_detail_value[item_detail] = no_sup(soup_detail_value[item_detail])
                    value_detail = soup_detail_value[item_detail].string if \
                            soup_detail_value[item_detail].string is not None else \
                                get_item(soup_detail_value[item_detail].children)
                    if name_str == '出品公司' and value_detail is not None:
                        data_csv_noRepeat.loc[item, '出品公司'] = \
                            str(value_detail).replace("\n", '').replace('，', '')
                        j += 1
                    if name_str == '导演' and value_detail is not None:
                        data_csv_noRepeat.loc[item, '导演'] = str(value_detail).replace("\n", '').replace('，', '')
                        j += 1
                    if name_str == '编剧' and value_detail is not None:
                        data_csv_noRepeat.loc[item, '编剧'] = str(value_detail).replace("\n", '').replace('，', '')
                        j += 1
                    if name_str == '制片人' and value_detail is not None:
                        data_csv_noRepeat.loc[item, '制片人'] = str(value_detail).replace("\n", '').replace('，', '')
                        j += 1
                    if name_str == '片长' and value_detail is not None:
                        data_csv_noRepeat.loc[item, '片长'] = str(value_detail).replace("\n", '').replace('，', '')
                        j += 1
                    if name_str == '票房' and value_detail is not None:
                        data_csv_noRepeat.loc[item, '票房'] = str(value_detail).replace("\n", '').replace('，', '')
                        j += 1
                    if j == 6:
                        continue
                movies += 1
        except Exception as e:
            pass
        pbar.update(1)
        pbar.set_postfix(**{'move_num':movies, 'j':j})
data_csv_noRepeat.to_csv('./data_csv_noRepeat_new.csv')

  0%|          | 105/23422 [01:05<4:01:09,  1.61it/s, j=0, move_num=95]


KeyboardInterrupt: 

In [98]:
data_csv_noRepeat

,types,rating,rank,id,regions,title,url,release_date,actor_count,vote_count,score,actors,cover_url,is_watched,编剧,片长,票房,出品公司,导演,制片人
0,"['犯罪', '剧情']","['9.7', '50']",1,1292052,['美国'],肖申克的救赎,https://movie.douban.com/subject/1292052/,1994-09-10,25,2604521,9.7,"['蒂姆·罗宾斯', '摩根·弗里曼', '鲍勃·冈顿', '威廉姆·赛德勒', '克兰西·...",https://img2.doubanio.com/view/photo/s_ratio_p...,False,弗兰克·德拉邦特,142 分钟,5830 万美元,NaN,NaN,NaN
1,"['剧情', '爱情', '同性']","['9.6', '50']",2,1291546,"['中国大陆', '中国香港']",霸王别姬,https://movie.douban.com/subject/1291546/,1993-07-26,26,1934038,9.6,"['张国荣', '张丰毅', '巩俐', '葛优', '英达', '蒋雯丽', '吴大维',...",https://img3.doubanio.com/view/photo/s_ratio_p...,False,NaN,171 分钟,NaN,汤臣（香港）电影有限公司,NaN,NaN
2,"['剧情', '喜剧', '爱情', '战争']","['9.6', '50']",3,1292063,['意大利'],美丽人生,https://movie.douban.com/subject/1292063/,2020-01-03,29,1202892,9.6,"['罗伯托·贝尼尼', '尼可莱塔·布拉斯基', '乔治·坎塔里尼', '朱斯蒂诺·杜拉诺'...",https://img2.doubanio.com/view/photo/s_ratio_p...,False,温琴佐·切拉米 / 罗伯托·贝尼尼,116 分钟,NaN,NaN,罗伯托·贝尼尼,NaN
3,"['剧情', '历史', '战争']","['9.6', '50']",4,1295124,['美国'],辛德勒的名单,https://movie.douban.com/subject/1295124/,1993-11-30,47,1003257,9.6,"['连姆·尼森', '本·金斯利', '拉尔夫·费因斯', '卡罗琳·古多尔', '乔纳森·...",https://img2.doubanio.com/view/photo/s_ratio_p...,False,斯蒂文·泽里安,195 分钟,约 3.21 亿美元,环球影业公司,史蒂文·斯皮尔伯格,布兰克·鲁斯蒂
4,"['剧情', '犯罪', '悬疑']","['9.6', '50']",5,1296141,['美国'],控方证人,https://movie.douban.com/subject/1296141/,1957-12-17,51,453853,9.6,"['泰隆·鲍华', '玛琳·黛德丽', '查尔斯·劳顿', '爱尔莎·兰切斯特', '约翰·...",https://img1.doubanio.com/view/photo/s_ratio_p...,False,NaN,116分钟,1000,Edward Small Productions（美国）,NaN,阿瑟·霍恩布洛
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23417,"['黑色电影', '犯罪', '惊悚']","['7.3', '40']",79,1302943,['美国'],合约杀手,https://movie.douban.com/subject/1302943/,1942-05-13,5,479,7.3,"['维罗妮卡·莱克', '罗伯特·普雷斯顿', '拉尔德·克雷加', '艾伦·拉德', '塔...",https://img9.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN,NaN,NaN,NaN,NaN
23418,"['犯罪', '剧情', '黑色电影', '恐怖']","['7.3', '40']",80,2033530,['美国'],涨潮小屋,https://movie.douban.com/subject/2033530/,1950-03-25,24,458,7.3,"['路易斯·海沃德', '李·鲍曼', '简·维亚特', '多萝西·帕特里克', '安·休梅...",https://img1.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN,NaN,NaN,NaN,NaN
23419,"['动画', '恐怖', '黑色电影']","['7.3', '40']",81,26877383,['日本'],成人童话系列1 令人恐怖的格林童话,https://movie.douban.com/subject/26877383/,2000-06-21,0,367,7.3,[],https://img2.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN,NaN,NaN,NaN,NaN
23420,"['黑色电影', '惊悚']","['7.2', '40']",82,1293416,['美国'],逃狱雪冤,https://movie.douban.com/subject/1293416/,1947-09-27,2,607,7.2,"['亨弗莱·鲍嘉', '劳伦·白考尔']",https://img9.doubanio.com/view/photo/s_ratio_p...,False,NaN,NaN,NaN,NaN,NaN,NaN


#### 通过盗版网站爬取信息

In [20]:
def get_cookie(url):
    # 声明一个CookieJar对象实例来保存cookie
    cookie = cookiejar.CookieJar()
    # 利用urllib.request库的HTTPCookieProcessor对象来创建cookie处理器,也就CookieHandler
    handler=request.HTTPCookieProcessor(cookie)
    # 通过CookieHandler创建opener
    opener = request.build_opener(handler)
    # 此处的open方法打开网页
    response = opener.open(url)
    # 打印cookie信息
    for item in cookie:
        Name = item.name
        Value = item.value
        # print(Name,Value)
        return Value

In [29]:


data_csv_noRepeat = pd.read_csv("./data_movies_noRepeat.csv")
us = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 "
      "Safari/537.36 Edg/100.0.1185.50",
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
      'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50']

headers = { "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36", 'cookie': '''PHPSESSID=4g5qom1ldfhkrqehsvjtivatfh; 6688_2359_120.228.2.45=1; 6688_2557_120.228.2.45=1; 6688_2581_120.228.2.45=1; 6688_2519_120.228.2.45=1; 6688_2602_120.228.2.45=1; 6688_2604_120.228.2.45=1; vrg_sc=1429993fc09e02be48ad9333730d834b; captcha_text=%E8%96%AA%2C%E5%8F%89%2C%E5%9E%82; BT_auth=93ecxylMO4UQ61SSjY5bnKjfZ5keLvMLZP53fIer4wApgqvZwezpg6XlCK4H5Zmz_GpNdBwSaRQxfg487WGPVJQTAGg3xzNdoYGWH2O0eNHhBukVXQ1sP_qK6w2apL7oTaaorWW3YrnjaejETeL6CEBaFN8x_W7AR-2isjL-Jx39jfcG; BT_cookietime=0b7dVaqyhKn8NKzTVpLMK4Q0shAvAmssN_aW8A79s3bU67cLoC7Q; beitouviews_6688=h3rryELJLqRfsP88aE7l9h8HfAE3zxlkHthSJDJMtpkObBxut%252BWVdpT45TsDqbplIZjHKSK2dZN6YsFDtO3mZzjn5qQE3KbNlZ%252BbJ5648s2baTqAYM98dWMuXV8NR8ZI1QDf6FxEOUZi%252F5jb0%252FlPeB8Wuw3B80zhrPN%252Fr9ATNvvJYGBFPK0Q%252FMcu4J7ZLstPfTaG0mnYsvaKH0%252FRwuRRlCCUTOUYuQRPYbcNYWW1nS4%252F%252BZitVAm4tXNJEyw4SQmc0VYAnY%252FYtNSyOVeL0q0UFf7jB5sK7nZKKOR30ALVoz0TYcABk2IFmOUTpHfxinq2w8M1GXPCU1Tppp20Vs49wYQjkEbZ6B4FkQUPVTd3Ft1JaniOLOAss%252B6xBdXfB3dqhsjLoHNWFXGVoU8L%252F2CowohXAjJR487Li1Z8Xef%252FMp4n0QBp9M%252FjlR8TViyQZ8x17coH9Tj%252FQxZHoMCYZD5K2z3QtU40NemUidisvJgJaJCv7skmGUO8p7ETqxEI1rG6WPKXynoiO%252BteOeGtfOiPAtZNigv39wMGBlIIp%252BwaK%252BpBaacb7LOmSt%252BrOKqMz14mbASzCmj2jhCkUSHiQ%252Fy6JgTXpGMuk1toUbQmmOH4LNrfh7Q5GZqzp26aEuySFxUAfBd%252FSnTkIfUGvjwx1eMB%252Fyq9Q5aTgWCY%252FcoxGbY%252BixE%253D''',
            'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'x-requested-with': 'XMLHttpRequest',
            'sec-fetch-site': 'same-origin',
            'origin': 'https://www.btnull.re',
            'referer': 'https://www.btnull.re/mv/----score--.html',
            'sec-ch-ua-platform': "macOS",
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors'}
movies = 0
baseUrl = 'https://www.btnull.re'
req = bs4.BeautifulSoup(requests.post(baseUrl, headers=headers).text.encode('utf-8'))
print(req)


<!DOCTYPE html>
<html lang="zh-cn">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width" name="viewport"/>
<title>跳转提示</title>
<style type="text/css">
*{ padding: 0; margin: 0; }
body{ background: #fff; font-family: '微软雅黑'; color: #333; font-size: 16px; }
.system-message{ padding: 24px 48px; }
.system-message h1{ font-size: 100px; font-weight: normal; line-height: 120px; margin-bottom: 12px; }
.system-message .jump{ padding-top: 10px}
.system-message .jump a{ color: #333;}
.system-message .success,.system-message .error{ line-height: 1.8em; font-size: 36px }
.system-message .detail{ font-size: 12px; line-height: 20px; margin-top: 12px; display:none}
</style>
</head>
<body>
<div class="system-message">
<p class="error">无权访问此页面，请登录！</p>
<p class="detail"></p>
<p class="jump">
页面自动 <a href="/user/login/" id="href">跳转</a> 等待时间： <b id="wait">3</b>
</p>
</div>
<script type="text/javascript">
(function(){
var wait = document.getElement